In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random

def scrape_goodreads_ratings(user_id, max_pages=10):
    """
    Scrape a user's star ratings from Goodreads.

    Args:
    - user_id (str): Goodreads user ID or profile suffix.
    - max_pages (int): Maximum number of pages to scrape (each page contains ~30 books).

    Returns:
    - pd.DataFrame: A DataFrame containing book titles and ratings.
    """
    base_url = f"https://www.goodreads.com/review/list/{user_id}?shelf=read"
    headers = {"User-Agent": "Mozilla/5.0"}
    books = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, "html.parser")

        # Find all book entries in the table
        rows = soup.find_all("tr", class_="bookalike review")
        if not rows:
            print("No more data found.")
            break

        for row in rows:
            try:
                # print(row)
                title = row.find("td", class_="field title").a.text.strip()
                rating_element = row.find("td", class_="field rating")
                rating = rating_element.find("span", class_="staticStars").get("title", "No rating")
                stars = map_rating(rating)

                author = row.find("td", class_="field author").find("div", class_="value").get_text(strip=True)
                isbn = row.find("td", class_="field isbn").find("div", class_="value").get_text(strip=True)
                num_ratings = row.find("td", class_="field num_ratings").find("div", class_="value").get_text(strip=True)
                avg_rating = row.find("td", class_="field avg_rating").find("div", class_="value").get_text(strip=True)
                num_pages = row.find("td", class_="field num_pages").find("div", class_="value").get_text(strip=True)
                review = row.find("td", class_="field review").find("div", class_="value").get_text(strip=True)
                date_started = row.find("td", class_="field date_started").find("div", class_="value").get_text(strip=True)
                date_read = row.find("td", class_="field date_read").find("div", class_="value").get_text(strip=True)
                date_added = row.find("td", class_="field date_added").find("div", class_="value").get_text(strip=True)
                format = row.find("td", class_="field format").find("div", class_="value").get_text(strip=True)
                date_pub_edition = row.find("td", class_="field date_pub_edition").find("div", class_="value").get_text(strip=True)


                books.append({"Title": title, "Author": author, "Rating": stars, "User_id": user_id, "isbn": isbn, "num_ratings":num_ratings,
                              "avg_rating":avg_rating, "num_pages": num_pages, "review": review, "date_started":date_started,
                              "date_read": date_read, "date_added":date_added, "format": format, "date_pub_edition": date_pub_edition})
#                 print(title, rating, stars)
            except AttributeError:
                # Handle rows with missing data
                continue

        print(f"Page {page} scraped successfully.")
        time.sleep(random.uniform(0,2))  # Be kind to the server and avoid being blocked

    # Return data as a pandas DataFrame
    return pd.DataFrame(books)



In [58]:
def map_rating(phrase):
    rating_map = {
        "liked it": 3,
        "really liked it": 4,
        "it was ok": 2,
        "it was amazing": 5,
        "did not like it": 1,
    }

    return rating_map.get(phrase, "Invalid rating")  # Default to "Invalid rating" if the phrase isn't in the dictionary


In [59]:
#df = pd.read_csv('goodreads_ratings.csv')
user_ids = []#list(df['User_id'].unique())

In [60]:
if __name__ == "__main__":
    for repeat in range(1000):
        user_id = random.randint(1, 200000000) #get a random user_id
        # user_id = 188067832 #This is my user id
        for add in range(0,100):

          print("User_id = ", user_id)
          if user_id in user_ids:
            print("Already have this user.")
            user_id += 1
            continue
          max_pages = 50  # Max number of pages to scrape, 20 books per page.
          ratings_data = scrape_goodreads_ratings(user_id, max_pages)

          if not ratings_data.empty:
              print("user_id = ", user_id)
  #             print(ratings_data.head())
  #             ratings_data.to_csv("goodreads_ratings.csv", index=False)
              ratings_data.to_csv('books_by_user.csv', mode='a', header=False, index=False)
              print("Data saved to books_by_user.csv.")
          else:
              print("No data retrieved.")
          user_id += 1


User_id =  160058167
No more data found.
No data retrieved.
User_id =  160058168
No more data found.
No data retrieved.
User_id =  160058169
Page 1 scraped successfully.
No more data found.
user_id =  160058169
Data saved to books_by_user.csv.
User_id =  160058170
No more data found.
No data retrieved.
User_id =  160058171
Page 1 scraped successfully.
No more data found.
user_id =  160058171
Data saved to books_by_user.csv.
User_id =  160058172
No more data found.
No data retrieved.
User_id =  160058173
Page 1 scraped successfully.
Page 2 scraped successfully.
Page 3 scraped successfully.
Page 4 scraped successfully.
Page 5 scraped successfully.
No more data found.
user_id =  160058173
Data saved to books_by_user.csv.
User_id =  160058174
No more data found.
No data retrieved.
User_id =  160058175
No more data found.
No data retrieved.
User_id =  160058176
Page 1 scraped successfully.
No more data found.
user_id =  160058176
Data saved to books_by_user.csv.
User_id =  160058177
No more

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv('books_by_user.csv')
df = df.drop_duplicates()
# Print the entire DataFrame
print(df)

duplicate_count = df['Title'].duplicated().sum()
print("Number of books with at least two people rating it:", duplicate_count)
duplicate_counts_per_value = df['Title'].value_counts()
print(duplicate_counts_per_value)
print("Number of unique books: ", df['Title'].nunique())
num_users = df['User_id'].nunique()
user_ids = list(df['User_id'].unique())
print("number of users is: ", num_users)
print("user_ids = ", user_ids)